In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error,mean_squared_error
from sklearn.inspection import permutation_importance

In [ ]:
raw_df = pd.read_csv('stats.csv')

In [ ]:
df = raw_df.copy()
df.drop('last_name, first_name',axis = 1,inplace=True)
df.drop('player_id',axis = 1,inplace=True)

In [ ]:
train_df = df[(df['year'] == 2021) | (df['year'] == 2020)]
val_df = df[df.year==2022]
test_df = df[df.year==2023]

## Linear Regression Training / Validation 
#### all features

In [4]:
####TRAINING
features = ['pa', 'hit', 'single', 'double', 'triple','home_run', 'k_percent', 'bb_percent',
      'exit_velocity_avg',
      'sweet_spot_percent', 'barrel_batted_rate',
      'solidcontact_percent', 'hard_hit_percent', 
      'avg_best_speed','avg_hyper_speed',
      'whiff_percent', 'swing_percent',
      'groundballs_percent', 'flyballs_percent']

X_train = train_df[features]
y_train = train_df['on_base_plus_slg']

model = linear_model.LinearRegression()
model.fit(X_train, y_train)
b = model.intercept_
w = model.coef_[0]
print(f'TRAINING:::intercept={b:.3}, slope={w:.3}')
print(f'Accuracy on Training Data R^2= {model.score(X_train,y_train):.3}')

In [ ]:
####VALIDATION
X_val = val_df[features]
y_val = val_df['on_base_plus_slg']

validation_predictions = model.predict(X_val)
MAE_val = mean_absolute_error(y_val,validation_predictions)
MSE_val = mean_squared_error(y_val,validation_predictions)
RMSE_val = np.sqrt(MSE_val)

print("MAE_val: %f" % (MAE_val))
print("RMSE_val: %f" % (RMSE_val))

## PERMUTATION FEATURE IMPORTANCE ON LINEAR REGRESSION MODEL
##### THERE ARE MANY FEATURES AND ALL OF THEM HAVE VERY SMALL WEIGHTS (COEF VALUES) SO I AM GOING TO PERFORM PERMUTATION IMPORTANCE
##### PERMUTATION IMPORTANCE::::evalutates the contributions of each feature to a fitted model 

In [ ]:

X_feat_importance = val_df[features]
y_feat_importance = val_df['on_base_plus_slg']
model = linear_model.LinearRegression()
model.fit(X_feat_importance, y_feat_importance)
pi = permutation_importance(estimator=model, X=X_feat_importance, y=y_feat_importance, random_state=0)
print(f'pi.importances_mean={pi.importances_mean}')
print(f'pi.importances_std={pi.importances_std}')
print(f'pi.importances={pi.importances}')
plt.bar(x=range(X_feat_importance.columns.size), height=pi.importances_mean, tick_label=X_feat_importance.columns)
plt.title('Feature importance for LinearRegression, on_base_plus_slg vs. rest of MLB')
plt.xlabel('feature name')
plt.xticks(rotation=45)
_ = plt.ylabel(r'reduction in $R^2$ on shuffling feature')

### ANALYZING SUBSETS OF THE MOST IMPORTANT FEATURES


#### 1) features: pa, hit, home_run, bb_percent

In [ ]:

X_train = train_df[['pa','hit','home_run','bb_percent']]
y_train = train_df['on_base_plus_slg']
model = linear_model.LinearRegression()
model.fit(X_train,y_train)
print(f'The accuracy on the training data is = {model.score(X_train,y_train):.3}.')

X_val = val_df[['pa','hit','home_run','bb_percent']]
y_val = val_df['on_base_plus_slg']
validation_predictions = model.predict(X_val)
MAE_val = mean_absolute_error(y_val,validation_predictions)
MSE_val = mean_squared_error(y_val,validation_predictions)
RMSE_val = np.sqrt(MSE_val)

print("MAE_val: %f" % (MAE_val))
print("RMSE_val: %f" % (RMSE_val))

#### 2) features: pa, single, double, triple, home_run, bb_percent
##### changing hit--> single,double,triple

In [ ]:
features = ['pa','single','double','triple','home_run','bb_percent']
X_train = train_df[features]
y_train = train_df['on_base_plus_slg']
model = linear_model.LinearRegression()
model.fit(X_train,y_train)
print(f'The accuracy on the training data is = {model.score(X_train,y_train):.3}.')

X_val = val_df[features]
y_val = val_df['on_base_plus_slg']
validation_predictions = model.predict(X_val)
MAE_val = mean_absolute_error(y_val,validation_predictions)
MSE_val = mean_squared_error(y_val,validation_predictions)
RMSE_val = np.sqrt(MSE_val)

print("MAE_val: %f" % (MAE_val))
print("RMSE_val: %f" % (RMSE_val))

## Testing Linear Regression on the subset or best model???
